In [185]:
import pandas as pd
import numpy as np

In [268]:
data=pd.read_csv('btcusdt_1h_zelta.csv')

In [269]:
data['close>open']=0
data['close/open']=data['close']/data['open']

In [270]:
data['close>open']=np.where(data['close']/data['open']>1,1,data['close>open'])

In [271]:
data

,datetime,open,high,low,close,volume,close>open,close/open
0,2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199,0,0.986392
1,2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006,0,0.975909
2,2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572,1,1.009633
3,2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030,1,1.005984
4,2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329,1,1.012358
...,...,...,...,...,...,...,...,...
35203,2022-01-12 01:30:00,42972.04,43095.26,42692.19,42800.38,1219.601780,0,0.996005
35204,2022-01-12 02:30:00,42797.62,42823.69,42643.74,42659.20,702.103800,0,0.996766
35205,2022-01-12 03:30:00,42664.71,42776.14,42597.41,42713.13,561.859930,1,1.001135
35206,2022-01-12 04:30:00,42713.12,42886.28,42633.97,42729.29,681.142010,1,1.000379


In [272]:
split_size=0.8
total_size=len(data)
train=data[:int(split_size*total_size)]
test=data[int(split_size*total_size):]

In [273]:
len(data)==len(train)+len(test)

True

In [192]:
x_train=train.drop(columns={'close','close/open','close>open'})
x_train.set_index('datetime',inplace=True)
y_train=train['close>open']
x_test=test.drop(columns={'close','close/open','close>open'})
x_test.set_index('datetime',inplace=True)
y_test=test['close>open']

In [193]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()
model.fit(x_train,y_train)


RandomForestClassifier()

In [194]:
pred=model.predict(x_test)

In [195]:
pred

array([0, 1, 1, ..., 1, 1, 1], dtype=int64)

In [196]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.64      0.54      0.58      3537
           1       0.60      0.70      0.64      3505

    accuracy                           0.62      7042
   macro avg       0.62      0.62      0.61      7042
weighted avg       0.62      0.62      0.61      7042



In [36]:
(pred==1).sum()

8982

In [35]:
(pred==0).sum()

1581

In [38]:
(y_test==1).sum()

5366

In [39]:
(y_test==0).sum()

5197

In [333]:
WINDOW_SIZE=6
bitcoin_prices=pd.DataFrame(data['close'])
bitcoin_prices.rename(columns={'close':'Price'},inplace=True)
bitcoin_prices_nbeats = bitcoin_prices.copy()
for i in range(WINDOW_SIZE):
  bitcoin_prices_nbeats[f"Price+{i+1}"] = bitcoin_prices_nbeats["Price"].shift(periods=i+1)
bitcoin_prices_nbeats.head()

,Price,Price+1,Price+2,Price+3,Price+4,Price+5,Price+6
0,13529.01,NaN,NaN,NaN,NaN,NaN,NaN
1,13203.06,13529.01,NaN,NaN,NaN,NaN,NaN
2,13330.18,13203.06,13529.01,NaN,NaN,NaN,NaN
3,13410.03,13330.18,13203.06,13529.01,NaN,NaN,NaN
4,13601.01,13410.03,13330.18,13203.06,13529.01,NaN,NaN


In [334]:
X = bitcoin_prices_nbeats.drop("Price", axis=1)
y = bitcoin_prices_nbeats["Price"]


split_size = int(len(X) * 0.8)
X_train, y_train = X[:split_size], y[:split_size]
X_test, y_test = X[split_size:], y[split_size:]
len(X_train), len(y_train), len(X_test), len(y_test)

(28166, 28166, 7042, 7042)

In [335]:
y_train

0        13529.01
1        13203.06
2        13330.18
3        13410.03
4        13601.01
           ...   
28161    54340.89
28162    53688.07
28163    54466.36
28164    54430.79
28165    54350.54
Name: Price, Length: 28166, dtype: float64

In [336]:

x_train=train.drop(columns={'close/open','close>open','datetime'})
y_train=train['close>open']
x_test=test.drop(columns={'close/open','close>open','datetime'})
y_test=test['close>open']

In [337]:
x_train=x_train[6:]
x_train

,open,high,low,close,volume
6,13539.00,13800.00,13510.00,13780.41,264.989684
7,13780.00,13818.55,13555.02,13570.35,292.188777
8,13569.98,13735.24,13400.00,13499.99,271.813553
9,13499.97,13670.00,13459.11,13616.99,213.095829
10,13632.00,13657.92,13497.98,13570.01,169.512196
...,...,...,...,...,...
28161,54387.91,54816.01,54299.95,54340.89,1519.751400
28162,54342.80,54669.35,53596.32,53688.07,2878.754265
28163,53691.25,54588.00,53511.39,54466.36,2069.621473
28164,54466.38,54647.87,54350.00,54430.79,1756.511006


In [338]:
X_train.dropna()

,Price+1,Price+2,Price+3,Price+4,Price+5,Price+6
6,13558.99,13601.01,13410.03,13330.18,13203.06,13529.01
7,13780.41,13558.99,13601.01,13410.03,13330.18,13203.06
8,13570.35,13780.41,13558.99,13601.01,13410.03,13330.18
9,13499.99,13570.35,13780.41,13558.99,13601.01,13410.03
10,13616.99,13499.99,13570.35,13780.41,13558.99,13601.01
...,...,...,...,...,...,...
28161,54387.91,54646.59,54452.60,55008.46,55364.69,55710.87
28162,54340.89,54387.91,54646.59,54452.60,55008.46,55364.69
28163,53688.07,54340.89,54387.91,54646.59,54452.60,55008.46
28164,54466.36,53688.07,54340.89,54387.91,54646.59,54452.60


In [339]:
x_train1=pd.concat([x_train,X_train],axis=1)

In [340]:
x_train1=x_train1[:28160]

In [341]:
x_train1

,open,high,low,close,volume,Price+1,Price+2,Price+3,Price+4,Price+5,Price+6
6,13539.00,13800.00,13510.00,13780.41,264.989684,13558.99,13601.01,13410.03,13330.18,13203.06,13529.01
7,13780.00,13818.55,13555.02,13570.35,292.188777,13780.41,13558.99,13601.01,13410.03,13330.18,13203.06
8,13569.98,13735.24,13400.00,13499.99,271.813553,13570.35,13780.41,13558.99,13601.01,13410.03,13330.18
9,13499.97,13670.00,13459.11,13616.99,213.095829,13499.99,13570.35,13780.41,13558.99,13601.01,13410.03
10,13632.00,13657.92,13497.98,13570.01,169.512196,13616.99,13499.99,13570.35,13780.41,13558.99,13601.01
...,...,...,...,...,...,...,...,...,...,...,...
28161,54387.91,54816.01,54299.95,54340.89,1519.751400,54387.91,54646.59,54452.60,55008.46,55364.69,55710.87
28162,54342.80,54669.35,53596.32,53688.07,2878.754265,54340.89,54387.91,54646.59,54452.60,55008.46,55364.69
28163,53691.25,54588.00,53511.39,54466.36,2069.621473,53688.07,54340.89,54387.91,54646.59,54452.60,55008.46
28164,54466.38,54647.87,54350.00,54430.79,1756.511006,54466.36,53688.07,54340.89,54387.91,54646.59,54452.60


In [342]:
y_train1=np.where(x_train1['close']/x_train1['open']>1/0.998,1,0)
y_train1=np.where(x_train1['close']/x_train1['open']<=0.998,-1,y_train1)
y_train1

array([ 1, -1, -1, ...,  1,  0,  0])

In [343]:
x_test1=pd.concat([x_test,X_test],axis=1)

In [344]:
x_test1

,open,high,low,close,volume,signals,signals,Price+1,Price+2,Price+3,Price+4,Price+5,Price+6
28166,54350.54,54580.00,53966.89,54122.07,1650.390799,0,0,54350.54,54430.79,54466.36,53688.07,54340.89,54387.91
28167,54127.12,54294.82,53783.23,54100.00,2037.761060,0,1,54122.07,54350.54,54430.79,54466.36,53688.07,54340.89
28168,54100.00,55000.00,53700.00,54662.79,3668.396569,1,1,54100.00,54122.07,54350.54,54430.79,54466.36,53688.07
28169,54661.64,55612.92,54606.00,55464.20,4769.800293,1,1,54662.79,54100.00,54122.07,54350.54,54430.79,54466.36
28170,55464.46,56400.13,55452.14,56280.41,4871.585812,1,1,55464.20,54662.79,54100.00,54122.07,54350.54,54430.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35203,42972.04,43095.26,42692.19,42800.38,1219.601780,1,1,42972.05,42755.00,42549.01,42705.24,41733.93,41642.00
35204,42797.62,42823.69,42643.74,42659.20,702.103800,1,1,42800.38,42972.05,42755.00,42549.01,42705.24,41733.93
35205,42664.71,42776.14,42597.41,42713.13,561.859930,1,1,42659.20,42800.38,42972.05,42755.00,42549.01,42705.24
35206,42713.12,42886.28,42633.97,42729.29,681.142010,1,1,42713.13,42659.20,42800.38,42972.05,42755.00,42549.01


In [345]:
y_test1=np.where(x_test1['close']/x_test1['open']>1/0.998,1,0)
y_test1=np.where(x_test1['close']/x_test1['open']<=0.998,-1,y_test1)
y_test1

array([-1,  0,  1, ...,  0,  0,  0])

In [346]:
x_train1=x_train1.drop('close',axis=1)
x_test1=x_test1.drop('close',axis=1)

In [347]:
(y_train1==0).sum()

11077

In [348]:
mode1=RandomForestClassifier()
mode1.fit(x_train1,y_train1)


RandomForestClassifier()

In [353]:
pred1=mode1.predict(x_test1)

In [354]:
from sklearn.metrics import classification_report
print(classification_report(pred1,y_test1))

              precision    recall  f1-score   support

          -1       0.43      0.48      0.46      2198
           0       0.35      0.36      0.36      1930
           1       0.54      0.47      0.50      2914

    accuracy                           0.44      7042
   macro avg       0.44      0.44      0.44      7042
weighted avg       0.45      0.44      0.45      7042



In [362]:
x_test1

,open,high,low,volume,Price+1,Price+2,Price+3,Price+4,Price+5,Price+6
28166,54350.54,54580.00,53966.89,1650.390799,54350.54,54430.79,54466.36,53688.07,54340.89,54387.91
28167,54127.12,54294.82,53783.23,2037.761060,54122.07,54350.54,54430.79,54466.36,53688.07,54340.89
28168,54100.00,55000.00,53700.00,3668.396569,54100.00,54122.07,54350.54,54430.79,54466.36,53688.07
28169,54661.64,55612.92,54606.00,4769.800293,54662.79,54100.00,54122.07,54350.54,54430.79,54466.36
28170,55464.46,56400.13,55452.14,4871.585812,55464.20,54662.79,54100.00,54122.07,54350.54,54430.79
...,...,...,...,...,...,...,...,...,...,...
35203,42972.04,43095.26,42692.19,1219.601780,42972.05,42755.00,42549.01,42705.24,41733.93,41642.00
35204,42797.62,42823.69,42643.74,702.103800,42800.38,42972.05,42755.00,42549.01,42705.24,41733.93
35205,42664.71,42776.14,42597.41,561.859930,42659.20,42800.38,42972.05,42755.00,42549.01,42705.24
35206,42713.12,42886.28,42633.97,681.142010,42713.13,42659.20,42800.38,42972.05,42755.00,42549.01


In [383]:
print(pred1)

[0 0 1 ... 1 1 1]


In [379]:
signals=pd.Series(0,index=x_test1.reset_index().index,name='signals')
position_open=False
trades=[]
for i in range(len(pred1)):
    if pred1[i]==1:
        if sum(trades)==0:
            position_open=True
            signals[i]=1
            trades.append(1)
        else:
            position_open=True
            signals[i]=0
    elif pred1[i]==-1:
        if sum(trades)==1:
            position_open=False
            signals[i]=-1
            trades.append(-1)
        else:
            position_open=False
            signals[i]=0
    elif pred1[i]==0:
        signals[i]==0
            
    
    